In [3]:
import os
import shutil 
import numpy as np
from PIL import Image
import sys
sys.path.append('../')
from torch.utils.data import DataLoader
from CustomDataclass import BrainMRIData
import torchvision.transforms as transforms 
import torch

Create subdirectories in VAL folder


In [6]:
trainPath = r'C:\Users\rrayn\OneDrive\Desktop\Personal-Project\Brain MRI Analysis\Brain-MRI-Analysis\Br35H-Mask-RCNN\TRAIN'
valPath = r'C:\Users\rrayn\OneDrive\Desktop\Personal-Project\Brain MRI Analysis\Brain-MRI-Analysis\Br35H-Mask-RCNN\VAL'
classes = ['yes', 'no']

for className in classes:
    os.makedirs(os.path.join(valPath, className), exist_ok=True)

Move some data from train to val

In [7]:
valPercentage = 0.2

for className in classes:
    trainClassPath = os.path.join(trainPath , className)
    valClassPath = os.path.join(valPath, className)
    allFiles = os.listdir(trainClassPath)
    np.random.shuffle(allFiles)
    splitIndex = int(valPercentage * len(allFiles))
    valFiles = allFiles[:splitIndex]
    for fileName in valFiles:
        src = os.path.join(trainClassPath, fileName)
        dst = os.path.join(valClassPath, fileName)
        shutil.move(src,dst)

Change image color channels to RGB 


In [4]:
trainPath = r'C:\Users\rrayn\OneDrive\Desktop\Personal-Project\Brain MRI Analysis\Brain-MRI-Analysis\Br35H-Mask-RCNN\TRAIN'
valPath = r'C:\Users\rrayn\OneDrive\Desktop\Personal-Project\Brain MRI Analysis\Brain-MRI-Analysis\Br35H-Mask-RCNN\VAL'
for root, dirs, files in os.walk(trainPath):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                img_path = os.path.join(root, file)
                img = Image.open(img_path)
                img_rgb = img.convert('RGB')
                img_rgb.save(img_path)  # Overwrite the original image

for root, dirs, files in os.walk(valPath):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                img_path = os.path.join(root, file)
                img = Image.open(img_path)
                img_rgb = img.convert('RGB')
                img_rgb.save(img_path)  # Overwrite the original image

Calculate mean and std for dataset to normalize pixel values between 0 and 1

In [4]:
trainPath = r'C:\Users\rrayn\OneDrive\Desktop\Personal-Project\Brain MRI Analysis\Brain-MRI-Analysis\Br35H-Mask-RCNN\TRAIN'
valPath = r'C:\Users\rrayn\OneDrive\Desktop\Personal-Project\Brain MRI Analysis\Brain-MRI-Analysis\Br35H-Mask-RCNN\VAL'
transform = transforms.Compose([transforms.Resize((200, 200)), transforms.ToTensor()])
    
    # Load the dataset using the custom BrainMRIData class
dataset = BrainMRIData(trainPath, transform=transform)
    
    # Create a DataLoader for the dataset
loader = DataLoader(dataset, batch_size=64, shuffle=False, num_workers=4)
    
    # Initialize variables to accumulate mean and standard deviation
mean = 0.0
std = 0.0
total_images = 0
    
    # Iterate over the DataLoader to calculate mean and standard deviation
for images, _ in loader:
        batch_mean = torch.mean(images, dim=[0, 2, 3])
        batch_std = torch.std(images, dim=[0, 2, 3])
        
        mean += batch_mean * images.size(0)
        std += batch_std * images.size(0)
        total_images += images.size(0)
    
    # Calculate final mean and std
mean /= total_images
std /= total_images
    
print(mean,std)


tensor([0.2802, 0.2804, 0.2806]) tensor([0.2664, 0.2664, 0.2665])
